## Mongo DB 

## SQL Triggers / scheduled events

MySQL Events are tasks that execute according to a specified schedule. Therefore, sometimes MySQL events are referred to as scheduled events.

MySQL Events are named object which contains one or more SQL statement. They are stored in the database and executed at one or more intervals.

**i.e.** you can create an event that optimizes all tables in the database that runs at 1:00 AM every Sunday.

```
SHOW PROCESSLIST;

``` 

if the event scheduler is not enabled (it should show up in the process list) , enable it with the following command

``` 
SET GLOBAL event_scheduler = ON;
```

```
SHOW PROCESSLIST;
``` 

now you should be able to see the event schedule in the process list 


### Sample mysql code 

create a new table messages that will hold the results for your scheduled event
```
CREATE TABLE messages (
    id INT PRIMARY KEY AUTO_INCREMENT,
    message VARCHAR(255) NOT NULL,
    created_at DATETIME NOT NULL
);
``` 

create event that inserts a message now into the database

```
CREATE EVENT IF NOT EXISTS test_event_01
ON SCHEDULE AT CURRENT_TIMESTAMP
DO
  INSERT INTO messages(message,created_at)
  VALUES('Test MySQL Event 1',NOW());
```

view the results 

``` 
SELECT * FROM messages;
```

show all registered events 

``` 
SHOW EVENTS FROM classicmodels;
```
by default mysql deletes the scheduled events that have already been executed, add the following clause to register the events. 
```
ON COMPLETION PERSERVE 
``` 


now let's try changing the execution time to add an offset of 1 minute
```
CREATE EVENT test_event_02
ON SCHEDULE AT CURRENT_TIMESTAMP + INTERVAL 1 MINUTE
ON COMPLETION PRESERVE
DO
   INSERT INTO messages(message,created_at)
   VALUES('Test MySQL Event 2',NOW());
``` 
check the result and show events register
```
SELECT * FROM messages;
SHOW EVENTS FROM classicmodels;
```

How would we write a scheduled event that would execute every minute of an hour? 
```
CREATE EVENT test_event_03
ON SCHEDULE EVERY 1 MINUTE
STARTS CURRENT_TIMESTAMP
ENDS CURRENT_TIMESTAMP + INTERVAL 1 HOUR
DO
   INSERT INTO messages(message,created_at)
   VALUES('Test MySQL recurring Event',NOW());
```

notice above how we can use starts and ends clauses to define a time interval , let's check the results

```
SELECT * FROM messages;

``` 

also we can drop an event just like any other thing we want to drop in sql 
```
DROP EVENT IF EXIST test_event_03;
```

## SQL Injection

SQL injection is a web security vulnerability that allows an attacker to interfere with the queries that an application makes to its database. It generally allows an attacker to view data that they are not normally able to retrieve.  

- SQL injection attack to compromise the underlying server or other back-end infrastructure, or perform a denial-of-service attack
- A successful SQL injection attack can result in unauthorized access to sensitive data, such as passwords, credit card details, or personal user information

### types of sql injection attacks 
- Retrieving hidden data, where you can modify an SQL query to return additional results.
- Subverting application logic, where you can change a query to interfere with the application's logic.
- UNION attacks, where you can retrieve data from different database tables.



### Retrieving hidden data using the WHERE clause

so imagine the following request is made: 
```
https://insecure-website.com/products?category=Gifts
```
in sql this is the corresponding request that is made: 
```
SELECT * FROM products WHERE category = 'Gifts' AND released = 1
```

The restriction released = 1 is being used to hide products that are not released. For unreleased products, presumably released = 0.

So an attacker can take advantage of this situation and do the following:

```
https://insecure-website.com/products?category=Gifts'-- 
```

This corresponds to the following sql query:

```
SELECT * FROM products WHERE category = 'Gifts'--' AND released = 1
```
The key thing here is that the double-dash sequence -- is a comment indicator in SQL, and means that the rest of the query is interpreted as a comment. This effectively removes the remainder of the query, so it no longer includes AND released = 1.

leaving the query to be logically equivalent to 

```
SELECT * FROM products WHERE category = 'Gifts
```

Which would show us all the unreleased products. 

Another variation on this would be to do the following:
```
https://insecure-website.com/products?category=Gifts'+OR+1=1--
```
which would correspond to the following query:
```
SELECT * FROM products WHERE category = 'Gifts' OR 1=1--' AND released = 1
```
Now this would retrieve all the Gifts from all categories because 1=1 is always true 



### Subverting app logic 

Consider an application that lets users log in with a username and password. If a user submits the username wiener and the password bluecheese, the application checks the credentials by performing the following SQL query:
```
SELECT * FROM users WHERE username = 'wiener' AND password = 'bluecheese'
```
If the query returns the details of a user, then the login is successful. Otherwise, it is rejected.

Here, an attacker can log in as any user without a password simply by using the SQL comment sequence -- to remove the password check from the WHERE clause of the query. For example, submitting the username administrator'-- and a blank password results in the following query:
```
SELECT * FROM users WHERE username = 'administrator'--' AND password = ''
```
This query returns the user whose username is administrator and successfully logs the attacker in as that user.

### Union Attacks (getting info from other tables)

In cases where the results of an SQL query are returned within the application's responses, an attacker can leverage an SQL injection vulnerability to retrieve data from other tables within the database. This is done using the UNION keyword, which lets you execute an additional SELECT query and append the results to the original query.

For example, if an application executes the following query containing the user input "Gifts":
```
SELECT name, description FROM products WHERE category = 'Gifts'
```
then an attacker can submit the input:

```

' UNION SELECT username, password FROM users--
```

This will cause the application to return all usernames and passwords along with the names and descriptions of products.

### python code that lends itself to sql injection attacks 

```
cursor.execute("SELECT admin FROM users WHERE username = '" + username + '");
cursor.execute("SELECT admin FROM users WHERE username = '%s' % username);
cursor.execute("SELECT admin FROM users WHERE username = '{}'".format(username));
cursor.execute(f"SELECT admin FROM users WHERE username = '{username}'");
```

how to fix these examples 
```
cursor.execute("SELECT admin FROM users WHERE username = %s'", (username, ));
cursor.execute("SELECT admin FROM users WHERE username = %(username)s", {'username': username});
``` 

Notice how strings are escaped or hace exact mappings making concatenation much more difficult 

Main Take away: try packaging sql statements as much as possible using bindings and prepared statements, avoid using string concatenation.

## SQL Transactions